# cloud-object-store-housekeeping

Housekeeping (delete, list, ...) on any S3 compliant Cloud Object Storage

In [ ]:
!pip install aiobotocore==1.4.1
!pip install botocore==1.10.62
!pip install s3fs==2021.11.0

In [ ]:
import logging
import os
import sys
import re
import s3fs

In [ ]:
access_key_id = os.environ.get('access_key_id')
secret_access_key = os.environ.get('secret_access_key')
endpoint = os.environ.get('endpoint')
bucket_name = os.environ.get('bucket_name')

# default: ls, operation in rm (delete), ls (list)
operation = os.environ.get('operation', 'ls')

# default: sysout, in "ls" operation, return list on sysout or as file (provide file name instead of sysout)
return_mode = os.environ.get('return_mode', 'sysout')

# file/folder to delete recoursively if operation == rm
delete_file = os.environ.get('delete_file')

In [ ]:
parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))

for parameter in parameters:
    logging.warning('Parameter: ' + parameter)
    exec(parameter)

In [ ]:
import configparser
config = configparser.ConfigParser()
config.read('ibm-sql-query.secrets')
config  = config['DEFAULT']
for key in config:  
    exec('{}="{}"'.format(key,config[key]))

In [ ]:
s3 = s3fs.S3FileSystem(
    anon=False,
    key=access_key_id,
    secret=secret_access_key,
    client_kwargs={'endpoint_url': endpoint}
)

In [ ]:
if operation == 'ls':
    l = s3.ls(bucket_name)
    if return_mode == 'sysout':
        print(l)
elif operation == 'rm':
    s3.rm(bucket_name + '/' + delete_file, recursive=True)
else:
    raise 'operation unknown: {}'.format(operation)